<a href="https://colab.research.google.com/github/Balasudeepreddy/Balasudeepreddy.github.io/blob/master/Copy_of_Copy_of_Cyka_Blyat_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [2]:
!pip install git+https://github.com/WillKoehrsen/feature-selector.git

  Cloning https://github.com/WillKoehrsen/feature-selector.git to /tmp/pip-req-build-y6q_e5v_
  Running command git clone -q https://github.com/WillKoehrsen/feature-selector.git /tmp/pip-req-build-y6q_e5v_
  Created wheel for feature-selector: filename=feature_selector-N_A-cp36-none-any.whl size=20711 sha256=07a646dce82fade834c9969d82ea2144b00e5251898fae6c26cbdf5093febaa9
  Stored in directory: /tmp/pip-ephem-wheel-cache-acczqi22/wheels/d5/67/b5/319a9229365dd4958877b18a9b31315c9f31e85c5a5bbcf7ff
Successfully built feature-selector


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [4]:
cd drive/My\ Drive/Kaggle

/content/drive/My Drive/Kaggle


In [5]:
ls

feature_f_importances.csv    test_f_encoded.csv    train_identity.csv
feature_importances.csv      test_identity.csv     train_transaction.csv
feature_low_importances.csv  test_transaction.csv
sample_submission.csv        train_f_encoded.csv


In [0]:
import os
import gc
import numpy as np
import pandas as pd

In [0]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [0]:
train_transaction = pd.read_csv('train_transaction.csv',index_col='TransactionID', error_bad_lines=False)
test_transaction = pd.read_csv('test_transaction.csv',index_col='TransactionID' , error_bad_lines=False)

train_identity = pd.read_csv('train_identity.csv',index_col='TransactionID' , error_bad_lines=False)
test_identity = pd.read_csv('test_identity.csv',index_col='TransactionID' , error_bad_lines=False)

In [9]:
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)

del train_transaction, train_identity
gc.collect()

test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)
del test_transaction, test_identity
gc.collect()

21

In [10]:
print(train.shape)
print(test.shape)

(590540, 433)
(506691, 432)


In [0]:
# Import label encoder 
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
le = preprocessing.LabelEncoder() 

In [0]:
cat_cols = ['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
            'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'ProductCD', 'card4', 'card6', 'M4','P_emaildomain',
            'R_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9',
            'P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3', 'R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']
for col in cat_cols:
    if col in train.columns:
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))   

In [0]:
train.replace(np.nan, -1, inplace=True)

In [14]:
train

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.500,4,4248,501,50,1,42,1,215,80,19.0,-1.0,32,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,-1.0,13.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,13.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
2987001,0,86401,29.000,4,9979,304,50,2,2,1,225,80,-1.0,-1.0,16,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
2987002,0,86469,59.000,4,11850,390,50,4,66,2,230,80,287.0,-1.0,36,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
2987003,0,86499,50.000,4,8796,467,50,2,17,2,376,80,-1.0,-1.0,55,32,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,-1.0,-1.0,-1.000000,-1.000000,84.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
2987004,0,86506,50.000,1,11687,414,50,2,2,1,320,80,-1.0,-1.0,16,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,...,0.0,70787.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,100.0,1,55,7,1,1,65,19,438,44,734,35,3,17,440,115,2,1,1,7,162,3,268,3,1,0,1,1,1,1565
2987005,0,86510,49.000,4,13098,455,50,4,126,2,172,80,36.0,-1.0,16,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
2987006,0,86522,159.000,4,2498,260,50,4,66,2,26,80,0.0,-1.0,55,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
2987007,0,86529,422.500,4,2917,390,50,4,126,2,225,80,-1.0,-1.0,29,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
2987008,0,86535,15.000,1,10024,0,50,4,126,2,237,80,-1.0,-1.0,1,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,...,-5.0,98945.0,-1.0,-1.0,0.0,-5.0,-1.0,-1.0,-1.0,-1.0,100.0,1,39,4,1,1,65,19,517,389,734,35,3,17,440,115,2,1,1,70,130,3,80,2,1,0,0,1,1,2693


In [0]:
test.replace(np.nan, -1, inplace=True)

In [16]:
test

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,...,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3663549,18403224,31.950,4,442,11,50,4,126,2,70,80,1.0,-1.0,16,32,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,-1.0,-1.0,-1.000000,-1.000000,418.0,203.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
3663550,18403263,49.000,4,11471,11,50,4,126,2,199,80,4.0,-1.0,2,32,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,-1.0,-1.0,-1.000000,-1.000000,231.0,634.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
3663551,18403310,171.000,4,11668,474,50,4,126,2,372,80,2635.0,-1.0,19,32,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,-1.0,-1.0,-1.000000,-1.000000,136.0,136.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
3663552,18403310,284.950,4,1069,260,50,4,66,2,105,80,17.0,-1.0,16,32,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,7.0,4.0,42.0,42.0,41.0,242.0,41.0,-1.0,-1.0,-1.000000,-1.000000,242.0,242.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
3663553,18403317,67.950,4,8674,352,50,2,17,2,164,80,6.0,-1.0,16,32,6.0,6.0,0.0,0.0,2.0,5.0,0.0,0.0,5.0,0.0,6.0,0.0,14.0,6.0,22.0,22.0,0.0,22.0,0.0,-1.0,-1.0,-1.000000,-1.000000,22.0,22.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
3663554,18403323,57.950,4,3072,221,50,4,126,2,412,80,-1.0,-1.0,16,32,5.0,5.0,0.0,0.0,2.0,3.0,0.0,0.0,2.0,0.0,4.0,0.0,10.0,4.0,36.0,36.0,35.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
3663555,18403350,87.000,4,7099,376,50,4,26,2,10,80,-1.0,-1.0,16,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
3663556,18403387,390.000,4,5479,70,50,2,2,1,94,80,303.0,-1.0,16,32,3.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,1.0,1.0,11.0,1.0,-1.0,-1.0,-1.0,126.0,4.0,-1.0,-1.0,-1.000000,-1.000000,126.0,126.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
3663557,18403405,103.950,4,10024,0,50,4,126,2,394,80,3.0,-1.0,16,32,152.0,148.0,0.0,0.0,135.0,95.0,0.0,0.0,77.0,0.0,122.0,0.0,407.0,108.0,128.0,128.0,13.0,644.0,13.0,-1.0,-1.0,-1.000000,-1.000000,106.0,631.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740


In [17]:
train=reduce_mem_usage(train)     
test=reduce_mem_usage(test)

Mem. usage decreased to 543.20 Mb (72.5% reduction)
Mem. usage decreased to 476.16 Mb (71.9% reduction)


In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split


In [25]:
def determine_feature_importance(df):
    #Determines the importance of individual features within a dataframe
    #Grab header for all feature values excluding score & ids
    features_list = df.columns.values[1::]
    print ("Features List: \n", features_list)

    #set X equal to all feature values, excluding Score & ID fields
    X = df.values[:,1:432]

    #set y equal to all Score values
    y = df.values[:,0]

    #fit a random forest with near-default paramaters to determine feature importance
    print ('\nCreating Random Forest Classifier...\n')
    forest = RandomForestClassifier(oob_score=True, n_estimators=100)
    print ('\nFitting Random Forest Classifier...\n')
    forest.fit(X,y)
    feature_importance = forest.feature_importances_
    print (feature_importance)
    
    #Make importances relative to maximum importance
    print ("\nMaximum feature importance is currently: ", feature_importance.max())
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    print ("\nNormalized feature importance: \n", feature_importance)
    print ("\nNormalized maximum feature importance: \n", feature_importance.max())
    print ("\nTo do: set fi_threshold == max?")
    print ("\nTesting: setting fi_threshhold == 1")
    fi_threshold=1

    #get indicies of all features over fi_threshold
    important_idx = np.where(feature_importance > fi_threshold)[0]
    print ("\nRetrieved important_idx: ", important_idx)

    #create a list of all feature names above fi_threshold
    important_features = features_list[important_idx]
    print ("\n", important_features.shape[0], "Important features(>", fi_threshold, "% of max importance:\n", important_features)

    #get sorted indices of important features
    sorted_idx = np.argsort(feature_importance[important_idx])[::-1]
    print ("\nFeatures sorted by importance (DESC):\n", important_features[sorted_idx])

    import matplotlib as plt
     #generate plot
    pos = np.arange(sorted_idx.shape[0]) + .5
    plt.subplot(1,2,2)
    plt.barh(pos,feature_importance[important_idx][sorted_idx[::-1]],align='center')
    plt.yticks(pos, important_features[sorted_idx[::-1]])
    plt.xlabel('Relative importance')
    plt.ylabel('Variable importance')
    plt.draw()
    plt.show()

    X = X[:, important_idx][:, sorted_idx]


    return "Feature importance determined"

IndentationError: ignored

In [24]:
determine_feature_importance(train)

Features List: 
 ['TransactionDT' 'TransactionAmt' 'ProductCD' 'card1' 'card2' 'card3'
 'card4' 'card5' 'card6' 'addr1' 'addr2' 'dist1' 'dist2' 'P_emaildomain'
 'R_emaildomain' 'C1' 'C2' 'C3' 'C4' 'C5' 'C6' 'C7' 'C8' 'C9' 'C10' 'C11'
 'C12' 'C13' 'C14' 'D1' 'D2' 'D3' 'D4' 'D5' 'D6' 'D7' 'D8' 'D9' 'D10'
 'D11' 'D12' 'D13' 'D14' 'D15' 'M1' 'M2' 'M3' 'M4' 'M5' 'M6' 'M7' 'M8'
 'M9' 'V1' 'V2' 'V3' 'V4' 'V5' 'V6' 'V7' 'V8' 'V9' 'V10' 'V11' 'V12' 'V13'
 'V14' 'V15' 'V16' 'V17' 'V18' 'V19' 'V20' 'V21' 'V22' 'V23' 'V24' 'V25'
 'V26' 'V27' 'V28' 'V29' 'V30' 'V31' 'V32' 'V33' 'V34' 'V35' 'V36' 'V37'
 'V38' 'V39' 'V40' 'V41' 'V42' 'V43' 'V44' 'V45' 'V46' 'V47' 'V48' 'V49'
 'V50' 'V51' 'V52' 'V53' 'V54' 'V55' 'V56' 'V57' 'V58' 'V59' 'V60' 'V61'
 'V62' 'V63' 'V64' 'V65' 'V66' 'V67' 'V68' 'V69' 'V70' 'V71' 'V72' 'V73'
 'V74' 'V75' 'V76' 'V77' 'V78' 'V79' 'V80' 'V81' 'V82' 'V83' 'V84' 'V85'
 'V86' 'V87' 'V88' 'V89' 'V90' 'V91' 'V92' 'V93' 'V94' 'V95' 'V96' 'V97'
 'V98' 'V99' 'V100' 'V101' 'V102' 'V103

NameError: ignored

In [27]:
train

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.500000,4,4248,501,50,1,42,1,215,80,19.0,-1.0,32,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,-1.0,13.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,13.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
2987001,0,86401,29.000000,4,9979,304,50,2,2,1,225,80,-1.0,-1.0,16,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
2987002,0,86469,59.000000,4,11850,390,50,4,66,2,230,80,287.0,-1.0,36,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
2987003,0,86499,50.000000,4,8796,467,50,2,17,2,376,80,-1.0,-1.0,55,32,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,-1.0,-1.0,-1.000000,-1.000000,84.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
2987004,0,86506,50.000000,1,11687,414,50,2,2,1,320,80,-1.0,-1.0,16,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,...,0.0,70787.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,100.0,1,55,7,1,1,65,19,438,44,734,35,3,17,440,115,2,1,1,7,162,3,268,3,1,0,1,1,1,1565
2987005,0,86510,49.000000,4,13098,455,50,4,126,2,172,80,36.0,-1.0,16,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
2987006,0,86522,159.000000,4,2498,260,50,4,66,2,26,80,0.0,-1.0,55,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
2987007,0,86529,422.500000,4,2917,390,50,4,126,2,225,80,-1.0,-1.0,29,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
2987008,0,86535,15.000000,1,10024,0,50,4,126,2,237,80,-1.0,-1.0,1,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,...,-5.0,98945.0,-1.0,-1.0,0.0,-5.0,-1.0,-1.0,-1.0,-1.0,100.0,1,39,4,1,1,65,19,517,389,734,35,3,17,440,115,2,1,1,70,130,3,80,2,1,0,0,1,1,2693


In [0]:
X=train.iloc[:,1:432]
y=train.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#using random forest classifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))